In [1]:
%cd /home/q123/Desktop/explo

import torch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "CartPole-v1"
kernel_name = "matern"
n_init = 3
n_steps = 1000

hypers = {
    'likelihood.noise_covar.noise': torch.tensor(10),
    'covar_module.kernel.base_kernel.lengthscale': torch.tensor(0.5),
    'covar_module.kernel.outputscale': torch.tensor(2.),
}

model,objective_env = setup_experiment(env_name,kernel_name,n_init)
#model.initialize(**hypers)
trainer = Trainer(model,objective_env,n_steps)
trainer.run(report_freq=100)

MathLog.src.helpers : WARNING : MLP dimensions : [4, 1]
current 22.0 / max 500.0 /batch_mean 23.079999923706055 /batch_max 500.0 
##############################
covar_lengthscale tensor([[0.3416]], grad_fn=<SoftplusBackward>)             covar_outputscale 6.766400337219238             noise 2736.8984375
##############################
current 9.0 / max 500.0 /batch_mean 25.829999923706055 /batch_max 500.0 
##############################
covar_lengthscale tensor([[0.3895]], grad_fn=<SoftplusBackward>)             covar_outputscale 7.633951663970947             noise 3532.574951171875
##############################
current 10.0 / max 500.0 /batch_mean 17.5 /batch_max 191.0 
##############################
covar_lengthscale tensor([[0.6733]], grad_fn=<SoftplusBackward>)             covar_outputscale 21.91843605041504             noise 2433.059326171875
##############################
current 9.0 / max 500.0 /batch_mean 22.59000015258789 /batch_max 500.0 
##############################
covar_

(tensor([0.0373, 0.1467, 0.3951, 0.9452, 0.0256]), tensor(500.))